best n_estimators: [0.9551672176827631, 0.9617357007040445, 0.9629886484974104, 0.9645678948901107, 0.9681745950174442, 0.9716901919134819, 0.9727181652591488, 0.9752238324454264, 0.976752545237565, 0.9769698111699242, 0.9771565856416056, 0.9776069913379128, 0.977047296024119, 0.977689957803016, 0.9790621877337535, 0.9798093427205921, 0.9794980329010855, 0.9798045463110473, 0.9802630602234899, 0.9806967355865037, 0.9813202117272214, 0.9817512604850084, 0.9817560568945533, 0.9821753430289327, 0.9830365840428023, 0.98325036686823, 0.9829163883036127, 0.9818579234972677, 0.9821927585635896, 0.9828105817930577, 0.9831266880221092, 0.9855345427137401, 0.9856521118477026, 0.9861724080830921, 0.9861806875995682, 0.9862912905196681, 0.986390987318065, 0.9861724080830921, 0.9860631184656057, 0.9862803843979648, 0.9861745778874098, 0.9863896740154512, 0.9866034568408791, 0.9872343559963686, 0.9873353660973786, 0.9871263796814956, 0.987235669298982, 0.9874459690174783, 0.9875373862993987, 0.98743

In [6]:
#coding:utf-8

import csv
import os
import shutil
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

g_ad_file_name = "/Users/check4068/Desktop/腾讯算法大赛/train_preliminary/ad.csv"



ad_data = pd.read_csv(g_ad_file_name)
ad_data = ad_data.loc[ad_data["product_id"] != "\\N"]
ad_data = ad_data.loc[ad_data["industry"] != "\\N"]

ad_data.head()

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
9,20,20,34647,5,312,267
14,27,27,59,1,62948,254
21,46,46,1263,1,1274,247
33,63,63,66,1,62928,242
34,65,64,4573,3,22,141


In [7]:
g_click_file_name = "/Users/check4068/Desktop/腾讯算法大赛/train_preliminary/click_log.csv"

click_data = pd.read_csv(g_click_file_name)
click_data = click_data.loc[click_data["click_times"] == 1]
click_data.head()   
len(click_data)

28426543

In [9]:
g_user_file_name = "/Users/check4068/Desktop/腾讯算法大赛/train_preliminary/user.csv"    
user_data = pd.read_csv(g_user_file_name)
user_data.head()

,user_id,age,gender
0,1,4,1
1,2,10,1
2,3,7,2
3,4,5,1
4,5,4,1


In [10]:
a = pd.merge(ad_data, click_data, on="creative_id")

a.head()
len(a)

15376393

In [11]:
b = pd.merge(a, user_data, on="user_id")
b.head()
len(b)

15376393

In [12]:
btarget = b[['age','gender']].values
print(btarget)


[[4 2]
 [4 2]
 [4 2]
 ...
 [5 2]
 [5 2]
 [4 2]]


In [13]:
blabel = b[['creative_id','ad_id','product_id','product_category','advertiser_id','industry','time','user_id','click_times']].values
print(blabel)

[[20 20 '34647' ... 78 827182 1]
 [967 1509 '124' ... 66 827182 1]
 [226621 203400 '23' ... 2 827182 1]
 ...
 [4443818 3810508 '9140' ... 91 425815 1]
 [4444878 3811453 '34304' ... 91 755982 1]
 [4444878 3811453 '34304' ... 91 839485 1]]


In [14]:
import lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import KFold
import numpy as np

from sklearn.model_selection import GridSearchCV


In [ ]:

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(btarget, blabel, test_size=0.3)
print(X_test)
print(y_test)
# 转换为Dataset数据格式
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)
# 参数
params = {
    'learning_rate': 0.1,
    'lambda_l1': 0.1,
    'lambda_l2': 0.2,
    'max_depth': 5,
    'objective': 'multiclass',  # 目标函数
    'num_class': 3,
}



# 模型训练
print('a')
gbm = lgb.train(params, train_data, valid_sets=[test_data])

# 模型预测
print('b')
y_pred = gbm.predict(X_test)
print(y_pred)
print('11111111111111111')
y_pred = [list(x).index(max(x)) for x in y_pred]
print(y_pred)

# 模型评估
print(accuracy_score(y_test, y_pred))
